In [ ]:
# ==========================================
# 📦 BLOCCO 1: Installazione e Import
# ==========================================

# Installa astroalign se necessario
!pip install astroalign

# Import delle librerie
from google.colab import files
from astropy.io import fits
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import astroalign as aa
from scipy import ndimage
from skimage import transform, registration, filters
import cv2
import io
from skimage.transform import resize  # Importa la funzione resize

print("✅ Librerie caricate con successo!")

# ==========================================
# 🛠️ BLOCCO 2: Funzioni di Supporto
# ==========================================

def load_image(filename, file_content):
    """Carica immagini FITS o PNG/JPG dai file uploadati"""
    try:
        if filename.lower().endswith(('.fits', '.fit')):
            # Per file FITS
            with io.BytesIO(file_content) as f:
                try:
                    with fits.open(f) as hdul:
                        data = hdul[0].data
                        if data.ndim > 2:
                            data = data[0]  # Usa il primo canale se 3D
                        return data.astype(np.float64)
                except Exception as e:
                    print(f"Errore durante l'apertura del file FITS: {e}")
                    return None
        else:
            # Per immagini comuni (PNG, JPG, etc.)
            try:
                if not file_content:
                    print("Errore: il contenuto del file è vuoto.")
                    return None

                img = Image.open(io.BytesIO(file_content)).convert('L')
                return np.array(img).astype(np.float64)
            except UnidentifiedImageError as e:
                print(f"Errore: Impossibile identificare il formato dell'immagine: {e}")
                return None
            except Exception as e:
                print(f"Errore durante l'apertura dell'immagine con PIL: {e}")
                return None
    except Exception as e:
        print(f"Errore generale durante il caricamento dell'immagine: {e}")
        return None

def normalize_image(img):
    """Normalizza l'immagine per migliorare il processing"""
    img_float = img.astype(np.float64)
    # Rimuovi valori estremi (outliers)
    p1, p99 = np.percentile(img_float, [1, 99])
    img_clipped = np.clip(img_float, p1, p99)
    # Normalizza tra 0 e 1
    img_norm = (img_clipped - img_clipped.min()) / (img_clipped.max() - img_clipped.min())
    return img_norm

def find_peaks_simple(image, threshold_factor=3.0, min_distance=10):
    """Rilevamento picchi semplificato per stelle"""

    # Applica filtro gaussiano per ridurre rumore
    smooth = ndimage.gaussian_filter(image, sigma=1.5)

    # Calcola soglia adattiva
    threshold = np.mean(smooth) + threshold_factor * np.std(smooth)

    # Trova i picchi locali con metodo semplice
    peaks = []
    h, w = image.shape

    for i in range(min_distance, h-min_distance):
        for j in range(min_distance, w-min_distance):
            if smooth[i, j] > threshold:
                # Verifica se è un massimo locale
                region = smooth[i-min_distance:i+min_distance+1, j-min_distance:j+min_distance+1]
                if smooth[i, j] == np.max(region):
                    peaks.append([i, j])

    return np.array(peaks)

def template_match_alignment(ref_img, target_img, template_size=200):
    """Usa template matching per allineare immagini simili"""

    h, w = ref_img.shape
    center_h, center_w = h//2, w//2

    # Estrai template dal centro dell'immagine di riferimento
    half_template = template_size // 2
    template = ref_img[center_h-half_template:center_h+half_template,
                      center_w-half_template:center_w+half_template]

    # Cerca il template nell'immagine target
    result = cv2.matchTemplate(target_img.astype(np.float32),
                              template.astype(np.float32),
                              cv2.TM_CCOEFF_NORMED)

    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    # Calcola lo spostamento
    shift_x = max_loc[0] - (center_w - half_template)
    shift_y = max_loc[1] - (center_h - half_template)

    return shift_x, shift_y, max_val

print("✅ Funzioni definite con successo!")

# ==========================================
# 📁 BLOCCO 3: Caricamento Immagini
# ==========================================

print("⬆️ Carica due immagini di galassia (prima quella di riferimento, poi quella da allineare)")
print("💡 Formati supportati: FITS, PNG, JPG, JPEG, TIFF")

uploaded = files.upload()

# Verifica che siano stati caricati esattamente 2 file
filenames = list(uploaded.keys())
if len(filenames) != 2:
    raise ValueError("⚠️ Devi caricare esattamente due immagini!")

print(f"📂 File caricati: {filenames}")

# Carica le immagini
print("📖 Caricamento immagini in corso...")
ref_img_content = uploaded[filenames[0]]
target_img_content = uploaded[filenames[1]]

ref_img = load_image(filenames[0], ref_img_content)
target_img = load_image(filenames[1], target_img_content)

if ref_img is None or target_img is None:
    raise ValueError("⚠️ Errore nel caricamento delle immagini. Controlla i file.")

print(f"📏 Dimensioni immagine di riferimento: {ref_img.shape}")
print(f"📏 Dimensioni immagine da allineare: {target_img.shape}")
print(f"📊 Range valori riferimento: {ref_img.min():.1f} - {ref_img.max():.1f}")
print(f"📊 Range valori target: {target_img.min():.1f} - {target_img.max():.1f}")
print("✅ Immagini caricate con successo!")

# ==========================================
# 🔄 BLOCCO 4: Normalizzazione
# ==========================================

print("\n🔄 Normalizzazione immagini in corso...")

# Normalizza le immagini
ref_img_norm = normalize_image(ref_img)
target_img_norm = normalize_image(target_img)

# Resize target image to match reference image dimensions
if target_img_norm.shape != ref_img_norm.shape:
    print("📏 Ridimensionamento immagine target...")
    target_img_norm = resize(target_img_norm, ref_img_norm.shape, anti_aliasing=True)
    print(f"   ✅ Ridimensionata a: {target_img_norm.shape}")

print("✅ Normalizzazione completata!")
print(f"📊 Range normalizzato: 0.0 - 1.0")

# ==========================================
# 👁️ BLOCCO 5: Visualizzazione Immagini Caricate
# ==========================================

print("\n👁️ Visualizzazione immagini caricate...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Prima riga - Immagini originali
axes[0, 0].imshow(ref_img, cmap='gray', vmin=np.percentile(ref_img, 1), vmax=np.percentile(ref_img, 99))
axes[0, 0].set_title(f"Riferimento - {filenames[0]}")
axes[0, 0].axis('off')

axes[0, 1].imshow(target_img, cmap='gray', vmin=np.percentile(target_img, 1), vmax=np.percentile(target_img, 99))
axes[0, 1].set_title(f"Da allineare - {filenames[1]}")
axes[0, 1].axis('off')

# Seconda riga - Immagini normalizzate
axes[1, 0].imshow(ref_img_norm, cmap='gray')
axes[1, 0].set_title("Riferimento (Normalizzata)")
axes[1, 0].axis('off')

axes[1, 1].imshow(target_img_norm, cmap='gray')
axes[1, 1].set_title("Target (Normalizzata)")
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# ==========================================
# ⭐ BLOCCO 6: Rilevamento Stelle Migliorato
# ==========================================

print("\n🔍 Tentativo con rilevamento stelle migliorato...")

alignment_success = False
alignment_method = ""
aligned_img = None

try:
    # Usa il metodo semplificato per trovare le stelle
    ref_stars = find_peaks_simple(ref_img_norm, threshold_factor=2.5, min_distance=15)
    target_stars = find_peaks_simple(target_img_norm, threshold_factor=2.5, min_distance=15)

    print(f"⭐ Stelle rilevate - Riferimento: {len(ref_stars)}, Target: {len(target_stars)}")

    if len(ref_stars) > 5 and len(target_stars) > 5:
        # Visualizza le stelle rilevate
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))

        axes[0].imshow(ref_img_norm, cmap='gray')
        if len(ref_stars) > 0:
            axes[0].scatter(ref_stars[:, 1], ref_stars[:, 0], s=100, c='red', marker='+', linewidths=3, alpha=0.8)
        axes[0].set_title(f"Riferimento ({len(ref_stars)} stelle rilevate)")
        axes[0].axis('off')

        axes[1].imshow(target_img_norm, cmap='gray')
        if len(target_stars) > 0:
            axes[1].scatter(target_stars[:, 1], target_stars[:, 0], s=100, c='red', marker='+', linewidths=3, alpha=0.8)
        axes[1].set_title(f"Target ({len(target_stars)} stelle rilevate)")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        # Prova astroalign con le stelle migliorate
        print("🤖 Tentativo AstroAlign con stelle rilevate...")
        try:
            aligned_img, footprint = aa.register(target_img_norm, ref_img_norm)
            print("🎉 AstroAlign funziona con le stelle migliorate!")
            alignment_success = True
            alignment_method = "AstroAlign con rilevamento migliorato"
        except Exception as e:
            print(f"❌ AstroAlign ancora non funziona: {str(e)[:100]}...")

    else:
        print("❌ Troppo poche stelle rilevate per AstroAlign")

except Exception as e:
    print(f"❌ Errore nel rilevamento stelle: {e}")

# ==========================================
# 🔄 BLOCCO 7: Template Matching
# ==========================================

if not alignment_success:
    print("\n🔄 Tentativo con Template Matching...")

    try:
        # Prova con diverse dimensioni di template
        template_sizes = [150, 200, 250, 300]
        best_confidence = 0
        best_result = None

        for template_size in template_sizes:
            try:
                shift_x, shift_y, confidence = template_match_alignment(ref_img_norm, target_img_norm, template_size)
                print(f"📏 Template {template_size}x{template_size}: spostamento=({shift_x:.1f}, {shift_y:.1f}), confidenza={confidence:.4f}")

                if confidence > best_confidence:
                    best_confidence = confidence
                    best_result = (shift_x, shift_y)

            except Exception as e:
                print(f"⚠️ Errore con template {template_size}: {e}")
                continue

        if best_confidence > 0.5:  # Soglia più permissiva
            shift_x, shift_y = best_result
            print(f"\n🎯 Miglior risultato: spostamento=({shift_x:.1f}, {shift_y:.1f}), confidenza={best_confidence:.4f}")

            # Applica la trasformazione
            tform = transform.SimilarityTransform(translation=(shift_x, shift_y))
            aligned_img = transform.warp(target_img_norm, tform.inverse,
                                       output_shape=ref_img_norm.shape)

            print("✅ Allineamento riuscito con Template Matching!")
            alignment_method = f"Template Matching (confidenza: {best_confidence:.3f})"
            alignment_success = True
        else:
            print(f"❌ Confidenza troppo bassa (massima: {best_confidence:.4f})")

    except Exception as e:
        print(f"❌ Errore nel Template Matching: {e}")

# ==========================================
# 🔄 BLOCCO 8: Correlazione di Fase (Backup)
# ==========================================

if not alignment_success:
    print("\n🔄 Tentativo con Correlazione di Fase...")

    try:
        # Usa solo il centro delle immagini per ridurre il rumore
        h, w = ref_img_norm.shape
        crop_ratio = 0.6  # Usa il 60% centrale
        crop_h = int(h * (1-crop_ratio) / 2)
        crop_w = int(w * (1-crop_ratio) / 2)

        ref_crop = ref_img_norm[crop_h:h-crop_h, crop_w:w-crop_w]
        target_crop = target_img_norm[crop_h:h-crop_h, crop_w:w-crop_w]

        shift, error, phasediff = registration.phase_cross_correlation(
            ref_crop, target_crop, upsample_factor=100
        )

        print(f"📐 Spostamento rilevato: ({shift[1]:.2f}, {shift[0]:.2f}) pixel")
        print(f"📊 Errore: {error:.6f}")

        if error < 0.5:  # Soglia più permissiva
            # Applica la trasformazione
            tform = transform.SimilarityTransform(translation=(-shift[1], -shift[0]))
            aligned_img = transform.warp(target_img_norm, tform.inverse,
                                       output_shape=ref_img_norm.shape)

            print("✅ Allineamento riuscito con Correlazione di Fase!")
            alignment_method = f"Correlazione di Fase (errore: {error:.4f})"
            alignment_success = True
        else:
            print("❌ Errore troppo alto nella correlazione di fase")

    except Exception as e:
        print(f"❌ Errore nella Correlazione di Fase: {e}")

# ==========================================
# 📊 BLOCCO 9: Visualizzazione Risultati
# ==========================================

if alignment_success:
    print(f"\n🎉 ALLINEAMENTO RIUSCITO!")
    print(f"🔧 Metodo utilizzato: {alignment_method}")

    # Calcola statistiche
    correlation = np.corrcoef(ref_img_norm.flatten(), aligned_img.flatten())[0, 1]
    mse = np.mean((ref_img_norm - aligned_img)**2)
    mae = np.mean(np.abs(ref_img_norm - aligned_img))

    # Visualizza i risultati completi
    fig, axes = plt.subplots(3, 3, figsize=(20, 16))

    # Prima riga - Immagini originali
    axes[0, 0].imshow(ref_img, cmap='gray', vmin=np.percentile(ref_img, 1), vmax=np.percentile(ref_img, 99))
    axes[0, 0].set_title("Riferimento (Originale)")
    axes[0, 0].axis('off')

    axes[0, 1].imshow(target_img, cmap='gray', vmin=np.percentile(target_img, 1), vmax=np.percentile(target_img, 99))
    axes[0, 1].set_title("Target (Originale)")
    axes[0, 1].axis('off')

    axes[0, 2].axis('off')  # Vuoto

    # Seconda riga - Immagini normalizzate
    axes[1, 0].imshow(ref_img_norm, cmap='gray')
    axes[1, 0].set_title("Riferimento (Normalizzata)")
    axes[1, 0].axis('off')

    axes[1, 1].imshow(target_img_norm, cmap='gray')
    axes[1, 1].set_title("Target (Normalizzata)")
    axes[1, 1].axis('off')

    axes[1, 2].imshow(aligned_img, cmap='gray')
    axes[1, 2].set_title("✅ Allineata")
    axes[1, 2].axis('off')

    # Terza riga - Analisi
    # Sovrapposizione colorata
    overlay = np.zeros((*ref_img_norm.shape, 3))
    overlay[:, :, 0] = ref_img_norm  # Rosso
    overlay[:, :, 1] = aligned_img   # Verde

    axes[2, 0].imshow(overlay)
    axes[2, 0].set_title("Sovrapposizione\n(Rosso + Verde = Giallo perfetto)")
    axes[2, 0].axis('off')

    # Differenza
    diff = np.abs(ref_img_norm - aligned_img)
    im_diff = axes[2, 1].imshow(diff, cmap='hot')
    axes[2, 1].set_title("Differenza\n(Nero = identico)")
    axes[2, 1].axis('off')
    plt.colorbar(im_diff, ax=axes[2, 1], shrink=0.8)

    # Statistiche
    stats_text = f"""QUALITÀ ALLINEAMENTO:

Correlazione: {correlation:.6f}
(1.000000 = perfetto)

MSE: {mse:.8f}
(0 = perfetto)

MAE: {mae:.8f}
(0 = perfetto)

Metodo: {alignment_method}

INTERPRETAZIONE:
• Correlazione > 0.99: Eccellente
• Correlazione > 0.95: Buono
• Correlazione > 0.90: Accettabile
• MSE < 0.001: Ottimo
• MSE < 0.01: Buono
"""

    axes[2, 2].text(0.05, 0.95, stats_text, transform=axes[2, 2].transAxes,
                    fontsize=10, verticalalignment='top', fontfamily='monospace',
                    bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))
    axes[2, 2].set_title("Statistiche Dettagliate")
    axes[2, 2].axis('off')

    plt.tight_layout()
    plt.show()

    # Stampa risultati
    print(f"\n📊 RISULTATI DETTAGLIATI:")
    print(f"   🎯 Correlazione: {correlation:.6f}")
    print(f"   📏 MSE (Mean Squared Error): {mse:.8f}")
    print(f"   📐 MAE (Mean Absolute Error): {mae:.8f}")
    print(f"   🔧 Metodo utilizzato: {alignment_method}")

    if correlation > 0.99:
        print("   🏆 QUALITÀ: ECCELLENTE!")
    elif correlation > 0.95:
        print("   ✅ QUALITÀ: BUONA")
    elif correlation > 0.90:
        print("   ⚠️  QUALITÀ: ACCETTABILE")
    else:
        print("   ❌ QUALITÀ: MIGLIORABILE")

else:
    print("\n❌ TUTTI I METODI AUTOMATICI SONO FALLITI")
    print("\n🖱️ METODO MANUALE NECESSARIO:")

    # Visualizza le immagini per selezione manuale
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))

    ax1.imshow(ref_img_norm, cmap='gray')
    ax1.set_title("RIFERIMENTO - Identifica stelle/punti distintivi")
    ax1.grid(True, alpha=0.3, color='red')

    ax2.imshow(target_img_norm, cmap='gray')
    ax2.set_title("TARGET - Trova gli STESSI punti")
    ax2.grid(True, alpha=0.3, color='red')

    plt.tight_layout()
    plt.show()

    print("\n📝 ISTRUZIONI PER ALLINEAMENTO MANUALE:")
    print("1. Identifica 3-4 stelle o punti distintivi comuni ad entrambe le immagini")
    print("2. Annota le coordinate (x, y) guardando il cursore del mouse")
    print("3. Sostituisci le coordinate nell'esempio sotto:")
    print()
    print("# ESEMPIO (sostituisci con le TUE coordinate):")
    print("ref_points = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]")
    print("target_points = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]")
    print()
    print("# Calcola trasformazione:")
    print("tform = transform.estimate_transform('similarity', target_points, ref_points)")
    print("aligned_img = transform.warp(target_img_norm, tform, output_shape=ref_img_norm.shape)")
    print("alignment_success = True")
    print("alignment_method = 'Manuale'")

# ==========================================
# 💾 BLOCCO 10: Salvataggio Risultati
# ==========================================

if alignment_success:
    print(f"\n💾 Salvataggio risultati...")

    try:
        # Prepara i nomi dei file
        base_name = filenames[0].split('.')[0]

        # 1. Salva immagine allineata (normalizzata)
        aligned_8bit = (aligned_img * 255).astype(np.uint8)
        plt.imsave(f"{base_name}_allineata.png", aligned_8bit, cmap='gray')

        # 2. Salva sovrapposizione colorata
        plt.imsave(f"{base_name}_sovrapposizione.png", overlay)

        # 3. Salva mappa differenze
        plt.imsave(f"{base_name}_differenze.png", diff, cmap='hot')

        # 4. Salva immagine allineata in scala originale
        if alignment_method.startswith("AstroAlign"):
            # Per AstroAlign, riapplica la trasformazione all'immagine originale
            try:
                aligned_original, _ = aa.register(target_img, ref_img)
                aligned_original_scaled = np.clip(aligned_original, 0, 65535).astype(np.uint16)
            except:
                # Fallback: scala l'immagine normalizzata
                aligned_original_scaled = (aligned_img * (target_img.max() - target_img.min()) + target_img.min())
                aligned_original_scaled = np.clip(aligned_original_scaled, 0, 65535).astype(np.uint16)
        else:
            # Per altri metodi, scala l'immagine normalizzata
            aligned_original_scaled = (aligned_img * (target_img.max() - target_img.min()) + target_img.min())
            aligned_original_scaled = np.clip(aligned_original_scaled, 0, 65535).astype(np.uint16)

        # Salva come FITS se l'originale era FITS
        if filenames[0].lower().endswith(('.fits', '.fit')):
            fits.writeto(f"{base_name}_allineata.fits", aligned_original_scaled, overwrite=True)
            print(f"✅ Salvato: {base_name}_allineata.fits")

        # 5. Salva report di qualità
        with open(f"{base_name}_report.txt", 'w') as f:
            f.write(f"REPORT ALLINEAMENTO IMMAGINI\n")
            f.write(f"="*50 + "\n\n")
            f.write(f"File di riferimento: {filenames[0]}\n")
            f.write(f"File da allineare: {filenames[1]}\n")
            f.write(f"Metodo utilizzato: {alignment_method}\n\n")
            f.write(f"STATISTICHE:\n")
            f.write(f"- Correlazione: {correlation:.6f}\n")
            f.write(f"- MSE: {mse:.8f}\n")
            f.write(f"- MAE: {mae:.8f}\n\n")
            f.write(f"INTERPRETAZIONE:\n")
            if correlation > 0.99:
                f.write("- Qualità: ECCELLENTE\n")
            elif correlation > 0.95:
                f.write("- Qualità: BUONA\n")
            elif correlation > 0.90:
                f.write("- Qualità: ACCETTABILE\n")
            else:
                f.write("- Qualità: MIGLIORABILE\n")

        print("✅ File salvati:")
        print(f"   - {base_name}_allineata.png (immagine allineata)")
        print(f"   - {base_name}_sovrapposizione.png (visualizzazione colorata)")
        print(f"   - {base_name}_differenze.png (mappa differenze)")
        print(f"   - {base_name}_report.txt (statistiche)")
        if filenames[0].lower().endswith(('.fits', '.fit')):
            print(f"   - {base_name}_allineata.fits (FITS allineato)")

        # Download automatico per Colab
        try:
            files.download(f"{base_name}_allineata.png")
            files.download(f"{base_name}_sovrapposizione.png")
            files.download(f"{base_name}_report.txt")
            if filenames[0].lower().endswith(('.fits', '.fit')):
                files.download(f"{base_name}_allineata.fits")
        except Exception as e:
            print(f"💡 File salvati nella directory di lavoro (download automatico non disponibile)")

    except Exception as e:
        print(f"⚠️ Errore nel salvataggio: {e}")

# ==========================================
# 🎯 BLOCCO 11: Riassunto Finale
# ==========================================

print(f"\n" + "="*80)
print("🎯 RIASSUNTO PROCESSO ALLINEAMENTO")
print("="*80)

print(f"📁 File processati:")
print(f"   - Riferimento: {filenames[0]} ({ref_img.shape})")
print(f"   - Target: {filenames[1]} ({target_img.shape})")

if alignment_success:
    print(f"\n✅ ALLINEAMENTO RIUSCITO!")
    print(f"🔧 Metodo: {alignment_method}")
    print(f"📊 Qualità: Correlazione = {correlation:.4f}")
    print(f"💾 File salvati e scaricati automaticamente")

    print(f"\n🔍 COSA SIGNIFICANO I RISULTATI:")
    print(f"• Sovrapposizione colorata: Giallo = perfetta corrispondenza")
    print(f"• Mappa differenze: Nero = identico, Colori caldi = diverse")
    print(f"• Correlazione > 0.99 = Eccellente, > 0.95 = Buona")
else:
    print(f"\n❌ ALLINEAMENTO AUTOMATICO FALLITO")
    print(f"💡 Possibili cause:")
    print(f"   - Troppo poche stelle visibili")
    print(f"   - Immagini troppo diverse")
    print(f"   - Poco overlap tra le immagini")
    print(f"   - Rumore eccessivo")
    print(f"\n🖱️  Prova l'allineamento manuale seguendo le istruzioni sopra")

print(f"\n💡 SUGGERIMENTI PER MIGLIORARE:")
print(f"• Usa immagini con molte stelle visibili")
print(f"• Assicurati che ci sia buona sovrapposizione")
print(f"• Pre-processa le immagini per migliorare il contrasto")
print(f"• Considera l'allineamento manuale per casi difficili")

print(f"\n🌌 Buona astrofotografia!")
print("="*80)